<a href="https://colab.research.google.com/github/FelixGonzalezp/proyecto_OCR/blob/main/Extracci%C3%B3n_de_informaci%C3%B3n_de_pagares_mediantes_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto: Extración de información mediante OCR

Motivo del proyecto:
Una de las princiales tareas que debo realizar es la correcta confección de las tablas de desarrollo asociadas a pagares. Estas tablas son confeccionas en una planilla excel, de información contenida en documentos en formato pdf, que generalmente corresponden a fotos escaneadas.

Esta información me llega vía email, y es cuando comienza el proceso de revisión.

Esta actividad tiene varias posibles fuentes de error:
1. Al extraer la información desde el pdf mediante inspección visual, pueden haber herrores de tipeo.
2. Generalmente, estos documentos se hacen en grandes cantidades, por lo que la revisión es tediosa y aumenta los focos de error, con el aumento de documentos.

Como somo solución, se propone utilizar un proceso de Reconocimiento Óptico de Caracteres (OCR utilizando sus siglas en inglés) que permita extraer la información desde los pagares. Con la información extraía se pueden general las tablas de desarrollo en un dataframe de pandas y compararlo con el adjunto que es enviado en los correos.

Como elemento adicional, los datos extraídos seran almacenados en una base de datos, para su consulta rápida mediante lengauje SQL. Lo que permetirá en proyectos furturos implementar soluciones ligadas a Store procedures o funciones en la base de datos.


In [ ]:
###### Importe de librerías para procesamiento de información

## Manejo de datos
import pandas as pd
import numpy as np
import copy

## Utilización de base de datos SQLite
import sqlite3
import urllib

## Manejo y edición de imagenes
import glob
import sys
import fitz
import cv2

## Librerías de OCR
from pytesseract import pytesseract
from PIL import Image

## Reconocimiento de expresiones regulares
import re

## Formateo de datos tipo date en pandas
from datetime import datetime, timedelta

## Formateo de datos en dataframe
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
## Se monta como fuente de datos carpeta de google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Con las principales librerías importadas que serán utilizadas en el proceso, a continuación se procede a describir el flujo de trabajo propuesto:

1. Convertir PDF en imagenes: Dado que los documentos corresponden a escaneos, la forma propuesta de trabajar es transformando cada una de las páginas del pdf en imagenes individuales.
2. Recortar imágenes: La información de interes se encuentra en la mitad superior de la primera hoja de cada uno de los pagares, por lo cual se recortarán las imagenes mantiniendo solo la zona superior. La ventaja de esto es que disminuye la cantidad de texto que se debe obtener y disminuye errores en la extración de información mediante expresiones regulares.
3. Configurar la librería Pytesseract para la extracción de texto
4. Mediante un ciclo recorrer las imagenes obtenidas para la extracción de información
5. Convertir a dataframe toda la información extraída con el fin de mejorar el manejo de información.
6. Establecer un punto de control: El proceso de OCR no esta excento de fallos, por lo cual antes de proseguir con los pasos es necesario establecer un punto que nos permita generar un control sanity y modificar/solucionar cualquier inconveniente con la información.


In [ ]:
#1. Convertir pdf en imagenes

#Definción de zoom para localizar la información de interes
zoom_x = 2.0  # horizontal zoom
zoom_y = 2.0  # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

#Ruta de los archivos
path = '/content/drive/MyDrive/Portafolio_proyectos/Extración de información mediante OCR/'
all_files = glob.glob(path + "*.pdf")

#Ciclo que recorre todos los archivos tipo pdf para la extración de sus respectivas páginas en imagenes
# Dado que solo nos intersa la primer hoja de los pagares, solo se guardan las páginas impares, comenzando desde 
# La página 1
contador=1
for filename in all_files:
    doc = fitz.open(filename)  # open document
    for page in doc:  # iterate through the pages
        if contador %2 != 0:
            pix = page.get_pixmap(matrix=mat, colorspace='GRAY')  # render page to an image
            pix.save(f'{path}pagina_{contador}.png')  # store image as a PNG
        contador +=1
doc.close()

In [ ]:
#2. Transformar las páginas del pdf en imagenes y recortar solo las zonas de interes 
all_pages = glob.glob(path + "*.png")

#Ccilo que recorre las imagenes para recortar las zonas de interes
for pagename in all_pages:
    #Open image with PIL
    img = cv2.imread(pagename)
    
    page_name_cut=pagename.split('\\')[-1].split('.')[0]+'_cut.png'
    
    crop_img = img[50:828, 0:1500]
    cv2.imwrite(page_name_cut, crop_img)
    


In [ ]:
#3. OCR
#Define path to tessaract.exe

path_to_tesseract = r'/usr/bin/tesseract'
pytesseract.tesseract_cmd = path_to_tesseract


In [ ]:
#4. Extraer información de los recortes
all_pages_cut = glob.glob(path + "*_cut.png")

################################### Diccionario ##########################################
## Se define una serie de diccionarios que permiten transformar las variables tipo texto en numerias
# Ejemplo: diciembre a 12 u octubre a 10.

lista_mes_nombre = ['enero','febrero','marzo','abril','mayo','junio',
                    'julio','agosto','septiembre','octubre','noviembre','diciembre']

lista_mes_numero = ['01','02','03','04','05','06',
                    '07','08','09','10','11','12']

lista_nombre = ['cero','uno','dos','tres','cuatro','cinco','seis','siete','ocho','nueve']

lista_numero = ['0','1','2','3','4','5','6','7','8','9']

listado_dia_semana = ['lunes', 'martes', 'mi[eé]rcoles','jueves','viernes','sabado','domingo']

dic_mes = dict(zip(lista_mes_nombre,lista_mes_numero))
dic_numero = dict(zip(lista_nombre,lista_numero))

###########################################################################################


################################### Listas para df ##########################################
#Listas vacías que almacenan la información extraida del texto para luego se transfromadas en dataframe

lista_numero_pagare = []
lista_moneda = []
lista_nominales = []
lista_fecha_emision =[]
lista_fecha_vencimiento = []
lista_base = []
lista_tasa_anualizada_compra = []

###########################################################################################

#Ciclo que recorre el texto extraído de las imagnes mediante el uso de expresiones regulares y coincidencias.
#Es importante descatar que para definir cada una de las expresiones regulares, primero es necesario 
#explorar adecuadamente el documento.

for pagename in all_pages_cut:
    #Open image with PIL
    img2 = cv2.imread(pagename)
    #Extract text from image
    text = pytesseract.image_to_string(img2)
    ##print(text)
    
    lista = text.split(' ')
    contador = 0
    guardar = []
    
    
    ##### Agregar nominales
    
    patron_nominales = re.compile('USD [0-9]+[.,]?.*')
    a = float(patron_nominales.findall(text.replace('\n',' '))[0].split(' ')[1].replace('.','').replace(',','.'))
    lista_nominales.append(a)
    lista_moneda.append('USD')
    
    
    ##### Agregar numero pagare
    
    patron_numero_pagare = re.compile('N°[ ]?[0-9]+[.,]?')
    b = int(patron_numero_pagare.findall(text.replace('\n',' '))[0][-2:])
    lista_numero_pagare.append(b)
    
    
    ##### Agregar fecha emisión 
    
    
    for i in lista_mes_nombre:
        patron_fecha_emision = re.compile(f'fecha de emisi[eéoó]n: [0-9]+ de {i} de [0-9]+')
    
        consulta = patron_fecha_emision.findall(text.replace('\n',' ').lower())
    
        if len(consulta)>0:
            tmp_1 = consulta
    

    c = tmp_1[0].split(' ')
    dia = c[3]
    mes = dic_mes[c[5]]
    anno = c[7]
    fecha_f = pd.to_datetime(anno+'-'+mes+'-'+dia, format="%Y-%m-%d")
    
    lista_fecha_emision.append(fecha_f)
    
    
    #### Agregar fecha vencimiento
    
    for i in listado_dia_semana:
        for j in lista_mes_nombre:
            patron_fecha_vencimiento = re.compile(f'pagado el {i}, [0-9]+ de {j} de [0-9]+')
        
            consulta =patron_fecha_vencimiento.findall(text.replace('\n',' ').lower())
        
            if len(consulta)>0:
                tmp_2 = consulta

    d = tmp_2[0].split(' ')

    dia = d[3]
    mes = dic_mes[d[5]]
    anno = d[7]
    fecha_f = pd.to_datetime(anno+'-'+mes+'-'+dia, format="%Y-%m-%d")
    lista_fecha_vencimiento.append(fecha_f)
    
    
    
    ##### Agregar tasa y base

    
    for i in lista:

        if i == 'anual,':
            lista_tasa_anualizada_compra.append(0.069)
            lista_base.append('Act/360')
        elif i == 'mensual,':
            lista_tasa_anualizada_compra.append(0.0696)
            lista_base.append('Act/360')
            

        contador +=1  

print('Finalizado!')

Finalizado!


In [ ]:
#5. Transformar la información recolectada en dataframe
dic = {
    'numero_pagare':lista_numero_pagare,
    'nominales': lista_nominales,
    'moneda':lista_moneda,
    'fecha_emision':lista_fecha_emision,
    'fecha_vencimiento':lista_fecha_vencimiento,
    'base':lista_base,
    'tasa_anualizada_compra':lista_tasa_anualizada_compra    
}
    
detalle_pagares=pd.DataFrame(dic)

detalle_pagares.sort_values(by='numero_pagare', inplace=True)

In [ ]:
#### PUNTO DE CONTROL ####
## Este punto nos permite verificar la integridad de la información extraída.

detalle_pagares

,numero_pagare,nominales,moneda,fecha_emision,fecha_vencimiento,base,tasa_anualizada_compra
0,78,2508967.010,USD,2022-09-25,2023-02-05,Act/360,0.070
1,79,356785.090,USD,2022-09-25,2023-02-15,Act/360,0.070


Se detecta que la fecha de emisión del pagaré 79 no es la correcta. Esto se debe a lo ya antes menconado sobre que el OCR no esta excento de errores. Para evitar esto, se deben utilizar imagenes con la mejor resolución posible o ingresar más puntos de control de errores en el reconocimiento de patrones.

En este caso, como la fecha es conocida, será corregida directamente

In [ ]:
detalle_pagares['fecha_emision'][1]='2022-10-30'

<ipython-input-9-aa2ae5689466>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_pagares['fecha_emision'][1]='2022-10-30'


In [ ]:
#### PUNTO DE CONTROL ####
## Este punto nos permite verificar la integridad de la información extraída.

detalle_pagares

,numero_pagare,nominales,moneda,fecha_emision,fecha_vencimiento,base,tasa_anualizada_compra
0,78,2508967.010,USD,2022-09-25,2023-02-05,Act/360,0.070
1,79,356785.090,USD,2022-10-30,2023-02-15,Act/360,0.070


Una vez revisada que la información extraida de los PDF es correcta y consistente se procede a crear y almacenar las tablas de desarrollo para su posterior comparación con los datos que vienen adjuntos en las ordenes de pagaré.

In [ ]:
#### Creación y almacenamiento de pagares para la comparación de tabla adjunta en correo

diccionario_pagares = {}
#### Crear EXCEL
for index, row in detalle_pagares.iterrows():
    
    
    ### Obtener datos desde tabla ###
    numero_pagare = row['numero_pagare']
    nominales = row['nominales']
    moneda = row['moneda']
    fecha_emision = pd.to_datetime(row['fecha_emision'],
                                   format="%Y-%m-%d")#datetime(2022,10,21)
    
    fecha_vencimiento = pd.to_datetime(row['fecha_vencimiento'],  
                                       format="%Y-%m-%d")#datetime(2022,11,18)
    base = row['base']
    tasa_anualizada_compra = row['tasa_anualizada_compra']
    
    ################## Parametros para creación de PAGARE ################################
    
    d={
        'numero_pagare':numero_pagare,
        'nominales':nominales,
        'moneda':moneda,
        'fecha_emision':fecha_emision.strftime("%Y-%m-%d"),
        'fecha_vencimiento':fecha_vencimiento.strftime("%Y-%m-%d"),
        'base':base,
        'tasa_anualizada_compra':tasa_anualizada_compra
               
    }
    
    parametros=pd.DataFrame(d, index=[0]).set_index('numero_pagare')

    
    
    ######################### Crear Tabla de desarollo ####################################
    lista_fechas = [(fecha_emision + timedelta(days=d)).strftime("%Y-%m-%d")
                for d in range((fecha_vencimiento - fecha_emision).days + 1)] 

    lista_interes = np.ones(len(lista_fechas))
    lista_interes[0]=0
    lista_interes = lista_interes*(tasa_anualizada_compra/360)*nominales

    dic={
        'dia':lista_fechas,
        'interes':lista_interes
        }

    pagare=pd.DataFrame(dic)#.set_index('dia')
    
    ### Crear columna de interes acumulado ###
    pagare['interes_acumulado']=pagare['interes'].cumsum()
    
    ### Crear columna de interes acumuadlo más capital ###
    pagare['saldo_capital']=pagare['interes_acumulado']+nominales
    
    diccionario_pagares[numero_pagare]=pagare

In [ ]:
### Se genera un ciclo que va explorando las planillas excel que contienen los pagares 
# y los compara con los creados con la información extraida directamente de los pagares
# El resultado se guarda en un nuevo df llamado resumen, que entrega la cantidad de errores encontrados
# Si todos los valores son 0, el pagaré enviado por correo esta correctamente creado

resumen = pd.DataFrame(columns=['pagare','fecha', 'interes', 'interes_acumulado', 'saldo_capital'])
nombre_columnas = ['fecha', 'interes', 'interes_acumulado', 'saldo_capital']
path_excel = '/content/drive/MyDrive/Portafolio_proyectos/Extración de información mediante OCR/'

for i in detalle_pagares['numero_pagare']:

    n_pagare = i

    pagare_revision = diccionario_pagares[i]

    pagare_correo = pd.read_excel(f'{path_excel}pagare {n_pagare}.xlsx', header=2, usecols='E:H').fillna(0)


    pagare_revision.columns=nombre_columnas
    pagare_correo.columns=nombre_columnas

    comparacion = round(pagare_revision,3) != round(pagare_correo,3)

    df_comparacion = pd.DataFrame(comparacion.sum()).T
    df_comparacion ['pagare']=n_pagare

    resumen = pd.concat([resumen,df_comparacion])

In [ ]:
resumen

,pagare,fecha,interes,interes_acumulado,saldo_capital
0,78,0,0,0,0
0,79,0,0,0,0


Una vez verificada la integridad de la información y que los pagarés fueron correctamente creados. Se procede a guadar la información contenida en ´detalle_pagares´ para su uso futuro

In [ ]:
#Creamos una conexión que nos permite crear un entorno de base de datos.
conn = sqlite3.connect("/content/drive/MyDrive/Portafolio_proyectos/Extración de información mediante OCR/pagare.db") #Creamos una base de datos a partir del método connect()
#Creamos un cursor que nos permite apuntar a la conección de nuestra Base de Datos
cursor = conn.cursor()
#Creamos un script SQL para abrir un archivo y luego leer
sqlScript = open("/content/drive/MyDrive/Portafolio_proyectos/Extración de información mediante OCR/ingreso_pagare.sqlite").read()
#Ejecutamos el cursor para abrir el archivo que contiene el script de creación/modificación de tablas
cursor.executescript(sqlScript)

In [ ]:
#Hacemos ua consulta a una base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #Seleccionamos la columna Nombre de la tabla sqlite_master, donde la columna type sea igual a 'table'
print(cursor.fetchall()) #Le pedimos a nuestro cursor que obtenga todos los resultados y los imprima:

[('Ingreso_pagare',)]


In [ ]:
# Se controla la correcta creación de la tabla ingreso_pagare
pd.read_sql("PRAGMA table_info('Ingreso_pagare')",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,numero_pagare,INTEGER,1,None,1
1,1,nominales,REAL,1,None,0
2,2,moneda,TEXT,1,None,0
3,3,fecha_emision,DATE,1,None,0
4,4,fecha_vencimiento,DATE,1,None,0
5,5,base,TEXT,1,None,0
6,6,tasa_anualizada_compra,REAL,0,None,0


In [ ]:
#Ejecutamos una consulta para insertar datos en la base de datos
for index, row in detalle_pagares.iterrows():
    
    
    ### Obtener datos desde tabla ###
    numero_pagare = row['numero_pagare']
    nominales = row['nominales']
    moneda = row['moneda']
    fecha_emision = pd.to_datetime(row['fecha_emision'],
                                   format="%Y-%m-%d")#datetime(2022,10,21)
    
    fecha_vencimiento = pd.to_datetime(row['fecha_vencimiento'],  
                                       format="%Y-%m-%d")#datetime(2022,11,18)
    base = row['base']
    tasa_anualizada_compra = row['tasa_anualizada_compra']
    cursor.execute(f" INSERT INTO ingreso_pagare VALUES ({numero_pagare}, {nominales}, '{moneda}', '{fecha_emision}', '{fecha_vencimiento}', '{base}',{tasa_anualizada_compra})")
    conn.commit() #Guardar datos

In [ ]:
    
pd.read_sql_query("SELECT * from ingreso_pagare",conn)
#print(df)

,numero_pagare,nominales,moneda,fecha_emision,fecha_vencimiento,base,tasa_anualizada_compra
0,78,2508967.010,USD,2022-09-25 00:00:00,2023-02-05 00:00:00,Act/360,0.070
1,79,356785.090,USD,2022-10-30 00:00:00,2023-02-15 00:00:00,Act/360,0.070


#Conclusiones finales del proyecto

* Se logra establecer un flujo de trabajo que permite extraer de manera automatizada los datos contenidos en los archivos PDF de los pagares
* Se logra comparar los pagaré generados en memoria con los adjutos, detectando 0 casos de diferencias.
* Se extablece un flujo que permite almacenar en una base de datos, la información extraída desde los PDF para su su uso futuro en otras implementaciones.